In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from itertools import product
import gc
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [37]:
test = pd.read_csv(r'../input/competitive-data-science-predict-future-sales/test.csv')
ic = pd.read_csv(r'../input/competitive-data-science-predict-future-sales/item_categories.csv')
items = pd.read_csv(r'../input/competitive-data-science-predict-future-sales/items.csv')
st = pd.read_csv(r'../input/competitive-data-science-predict-future-sales/sales_train.csv')
shops = pd.read_csv(r'../input/competitive-data-science-predict-future-sales/shops.csv')
ss = pd.read_csv(r'../input/competitive-data-science-predict-future-sales/sample_submission.csv')

In [38]:
test

In [39]:
ic

In [40]:
items

In [41]:
st

In [42]:
ss

In [43]:
#смотрим на рапределение item_cnt_day и item_price
fig, axes = plt.subplots(2, 1)
plt.subplots_adjust(hspace=0.5)

flierprops = dict(marker='o', markerfacecolor='cornflowerblue', markersize=6, markeredgecolor='navy')

_ = axes[0].boxplot(x=st.item_cnt_day, flierprops=flierprops, vert=False)
_ = axes[1].boxplot(x=st.item_price, flierprops=flierprops, vert=False)

_ = axes[0].set_title('item_cnt_day')
_ = axes[1].set_title('item_price')

In [44]:
# Извлечь данные больше, чем 0
st = st[st['item_price'] > 0]
# Извлечь данные меньше, чем 50000
st = st[st['item_price'] < 50000]
# Извлечь данные больше, чем 0
st = st[st['item_cnt_day'] > 0]
# Извлечь данные меньше, чем 51,000
st = st[st['item_cnt_day'] < 1000]

In [45]:
#  Меняем shop_id
st.loc[st['shop_id'] == 0, 'shop_id'] = 57
st.loc[st['shop_id'] == 1, 'shop_id'] = 58
st.loc[st['shop_id'] == 10, 'shop_id'] = 11
st.loc[st['shop_id'] == 39, 'shop_id'] = 40

test.loc[test['shop_id'] == 0, 'shop_id'] = 57
test.loc[test['shop_id'] == 1, 'shop_id'] = 58
test.loc[test['shop_id'] == 10, 'shop_id'] = 11
test.loc[test['shop_id'] == 39, 'shop_id'] = 40

In [46]:
test

In [47]:
shops

In [48]:
ss

In [49]:
# убираем item_name
items = items.drop(['item_name'], axis=1)
# когда продукт впервые был продан
items['first_sale_date'] = st.groupby('item_id').agg({'date_block_num': 'min'})['date_block_num']
items.head()
# меняем NaN на 34
items['first_sale_date'] = items['first_sale_date'].fillna(34)

In [50]:
test.isnull().sum()

In [51]:
ic.isnull().sum()

In [52]:
items.isnull().sum()

In [53]:
shops.isnull().sum()

In [54]:
st.isnull().sum()

In [55]:
#находим сумму за день
sum_per_day = st[['date', 'date_block_num', 'shop_id', 'item_id', 'item_cnt_day']]
sum_per_day = sum_per_day.groupby(['date'], as_index=False)
sum_per_day = sum_per_day.agg({'item_cnt_day':['sum']})

In [56]:
sum_per_day

In [57]:
#дневные продажи
daily_sales = go.Scatter(x=sum_per_day['date'], y=sum_per_day['item_cnt_day']['sum'])
layout = go.Layout(title='Daily sales', xaxis=dict(title='Date'), yaxis=dict(title='Sales'))
fig = go.Figure(data=[daily_sales], layout=layout)
iplot(fig)

In [58]:
#месячные продажи
monthly_per_shop = st[['date', 'date_block_num', 'shop_id', 'item_id', 'item_cnt_day']]
monthly_per_shop = monthly_per_shop.groupby(['date_block_num','shop_id'], as_index=False)
monthly_per_shop = monthly_per_shop.agg({'item_cnt_day':['sum']})
monthly_per_shop['item_cnt_month']  = monthly_per_shop['item_cnt_day']['sum']
monthly_per_shop = monthly_per_shop.drop(['item_cnt_day'], axis=1)
monthly_per_shop.head()

In [59]:
#продажи по месяцу
store_monthly_sales = []
for store in np.sort(monthly_per_shop['shop_id'].unique()):
    current_store_monthly_sales = monthly_per_shop[(monthly_per_shop['shop_id'] == store)]
    store_monthly_sales.append(go.Scatter(x=current_store_monthly_sales['date_block_num'], y=current_store_monthly_sales['item_cnt_month'], name=('Store %s' % store)))

layout = go.Layout(title='Stores monthly sales', xaxis=dict(title='Months'), yaxis=dict(title='Sales'))
fig = go.Figure(data=store_monthly_sales, layout=layout)
iplot(fig)

In [60]:
train = []
for i in st['date_block_num'].unique():
    all_shop = st.loc[st['date_block_num']==i, 'shop_id'].unique()
    all_item = st.loc[st['date_block_num']==i, 'item_id'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))
features = ['date_block_num', 'shop_id', 'item_id'] 
train = pd.DataFrame(np.vstack(train), columns=features)

In [61]:
group = st.groupby(features).agg({'item_cnt_day': 'sum',
                                               'item_price': 'mean'})
group = group.reset_index()
group = group.rename(columns={'item_cnt_day': 'item_cnt_month', 'item_price': 'item_price_mean'})

train = train.merge(group, on=features, how='left')

train.head()

In [62]:
# Добавить функцию количества проданных товаров
group = st.groupby(features).agg({'item_cnt_day': 'sum'})
group = group.reset_index()
group = group.rename(columns={'item_cnt_day': 'item_count'})

train = train.merge(group, on=features, how='left')

# убрать мусор
del group, st
gc.collect()

train.head()

In [63]:
# для test сделаем значение 34.
test['date_block_num'] = 34
union =  pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=features)
union = union.fillna(0)
union = union.fillna(0)

union.head()

In [64]:
#обьединяем таблицы
union = union.merge(shops, how = 'left', left_on = 'shop_id', right_on = 'shop_id')

In [65]:
union = union.merge(items, how = 'left', left_on = 'item_id', right_on = 'item_id')

In [66]:
union.isnull().sum()

In [67]:
union = union.merge(ic, how = 'left', left_on = 'item_category_id', right_on = 'item_category_id')

In [68]:
union.isnull().sum()

In [69]:
union['date_block_num'].unique()

In [70]:
union

In [71]:
#выделяем город
union['city'] = union.shop_name.str.split(' ').map(lambda x: x[0])
label_encoder = LabelEncoder()
union['city'] = label_encoder.fit_transform(union['city'])
#выделяем категорию магазина
union["shop_category"] = union.shop_name.str.split(" ").map(lambda x: x[1])
union['shop_category'] = label_encoder.fit_transform(union['shop_category'])
union = union.drop('shop_name', axis=1)
#категория товара
union['category'] = union['item_category_name'].apply(lambda x: x.split()[0])
union['category'] = label_encoder.fit_transform(union['category'])
union = union.drop('item_category_name', axis=1)
#выделяем месяц
union['month'] = (union['date_block_num'] ) %12 + 1

In [72]:
union['month'].unique()

In [73]:
union

In [74]:
#матрица
plt.figure(figsize=(20,12))
corrMatrix = union.corr()
sns.heatmap(corrMatrix, annot=True, vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')
plt.show()

In [75]:
union.info()

In [76]:
union.isnull().sum()

In [77]:
union['date_block_num'].unique()

In [78]:
X_train = union.drop(['item_cnt_month'], axis=1)
X_valid = union.drop(['item_cnt_month'], axis=1)
X_test = union.drop(['item_cnt_month'], axis=1)

y_train = union['item_cnt_month']
y_valid = union['item_cnt_month']

In [79]:
X_test

In [80]:
# Train 
X_train = union[union['date_block_num'] < 33]
X_train = X_train.drop(['item_cnt_month'], axis=1)
# Valid 
X_valid = union[union['date_block_num'] == 33]
X_valid = X_valid.drop(['item_cnt_month'], axis=1)
# Test 
X_test = union[union['date_block_num'] == 34]
X_test = X_test.drop(['item_cnt_month'], axis=1)


y_train = union[union['date_block_num'] < 33]['item_cnt_month']

y_valid = union[union['date_block_num'] == 33]['item_cnt_month']

del union
gc.collect();

In [81]:
y_train_pred = []
y_val_pred = []
y_test_pred = []

In [82]:
X_test

In [83]:
X_train

In [84]:
y_train

In [85]:
X_valid

In [86]:
y_valid

In [87]:
# lgb hyper-parameters
params = {'metric': 'rmse',
          'num_leaves': 256,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise' : True,
          'random_state': 10}

lgb_features = ['shop_id', 'city', 'item_category_id', 'category', 'month']


dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)
 
# тренировать LightGBM model
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=2400,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=200,
                      categorical_feature=lgb_features,
                      verbose_eval=100)    

In [88]:
X_test

In [89]:
yhat_train_lgb = lgb_model.predict(X_train).clip(0,20)
yhat_valid_lgb = lgb_model.predict(X_valid).clip(0,20)
yhat_test_lgb = lgb_model.predict(X_test).clip(0,20)

In [90]:
y_train_pred.append(yhat_train_lgb)
y_val_pred.append(yhat_valid_lgb)
y_test_pred.append(yhat_test_lgb)

In [91]:
from xgboost import XGBRegressor, plot_importance
xgb = XGBRegressor(seed = 13, 
    tree_method = "hist", #gpu_hist
    subsample = 0.9,
    n_estimators=500,
    max_depth = 10,
    learning_rate = 0.1,
    gamma = 10,
    colsample_bytree = 0.9
    )

xgb.fit(
    X_train,y_train,
    eval_metric="rmse",
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=True,
    early_stopping_rounds = 50
    )

In [92]:
results = xgb.evals_result()

plt.figure(figsize=(10,7))
plt.plot(results["validation_0"]["rmse"], label="Training loss")
plt.plot(results["validation_1"]["rmse"], label="Validation loss")
plt.axvline(21, color="gray", label="Optimal tree number")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()

In [93]:
yhat_train_xgb = xgb.predict(X_train).clip(0, 20)
yhat_val_xgb = xgb.predict(X_valid).clip(0, 20)
print('Valdation RMSE:', mean_squared_error(y_valid, yhat_val_xgb, squared=False)) 
yhat_test_xgb = xgb.predict(X_test).clip(0, 20)

In [94]:
# Добавить результаты для суммирования
y_train_pred.append(yhat_train_xgb)
y_val_pred.append(yhat_val_xgb)
y_test_pred.append(yhat_test_xgb)

In [95]:
#Преимущество  blending model заключается в том, что она сводит к минимуму риск переобучения и обеспечивает более надежное решение.
meta_model = LinearRegression()

In [96]:
def stacking_preds(meta_model, train_preds, val_preds, test_preds, y_train, y_val):
    stacked_train_predictions=np.column_stack(train_preds)
    stacked_val_predictions=np.column_stack(val_preds)
    stacked_test_predictions=np.column_stack(test_preds)
    
    #подгонка метамодели к сложенным прогнозам
    meta_model.fit(stacked_train_predictions, y_train)
    print('RMSE =',mean_squared_error(meta_model.predict(stacked_val_predictions).clip(0, 20), y_val, squared=False))
    return meta_model.predict(stacked_test_predictions).clip(0, 20)

In [97]:
stacking_preds

In [98]:
test_preds = stacking_preds(meta_model, y_train_pred, y_val_pred, y_test_pred, y_train, y_valid)

In [99]:
#1.22198
ss['item_cnt_month'] = test_preds
ss.to_csv('submission.csv', index=False)